In [ ]:
%reload_ext autoreload
%autoreload 2
import os 
import sys
import pandas as pd
import plotly 
import plotly.graph_objects as go

sys.path.insert(0, "./../../src/")
from data_loader import load_data

## Loading the data

In [ ]:
seasons = load_data()

In [ ]:
# tmp_df = pd.read_csv("./../../data/raw/2017-18_pbp.csv")
# display(tmp_df)
display(seasons)

In [ ]:
seasons_subset = seasons[seasons["season_name"] == "2018-19"]
display(seasons_subset.head())

## Visualizations 
Below you find a vizualization for each of the features found in the data  set to 

In [ ]:
# Histogram of event types
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=seasons["EVENTMSGTYPE"])
)
fig.update_layout(
    title_text="Histogram of Event Types", 
    xaxis_title="Event type name",
    yaxis_title="Frequency"
)
fig.show()

In [ ]:
# Histogram of event action types
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=seasons["EVENTMSGACTIONTYPE"])
)
fig.update_layout(
    title_text="Histogram of Event Action Types", 
    xaxis_title="Event action type name",
    yaxis_title="Frequency"
)
fig.show()

In [ ]:
# Number of events per game 
events_per_game_count = seasons_subset.groupby("GAME_ID").count()["EVENTNUM"]

# Histogram of event action types
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=events_per_game_count)
)
fig.update_layout(
    title_text="Histogram of Event Action Types", 
    xaxis_title="Event action type name",
    yaxis_title="Frequency"
)
fig.show()